In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
from scipy.interpolate import interp1d
from units import un
from newportmotorpy import NewportMotor  # your class from previous convos
import nidaqmx

# ─── User parameters ───────────────────────────────────────────────────────────
COM_PORT       = 'COM3'
AXIS           = 1
START_MM       = -0.5
STOP_MM        = 0.5
VELOCITY_MM_S  = 0.1
POLL_INTERVAL  = 0.01      # how often we poll motor position (s)

DAQ_CHANNEL    = "Dev2/ai0"
DAQ_RATE_HZ    = 1e5       # samples per second
POINTS_PER_SCAN= 5000      # total number of samples per sweep

# carrier estimate (for peak spacing)
CARRIER_FREQ   = 300e12    # 300 THz
PROMINENCE     = 0.05      # adjust to taste

# smoothing
GAUSS_SIGMA    = 30        # in samples

# ─── Helper functions ─────────────────────────────────────────────────────────
def compute_fwhm(x, y):
    y_norm = y / np.max(y)
    half = 0.5
    mask = y_norm >= half
    idx = np.where(mask)[0]
    if len(idx)<2:
        return np.nan, np.nan, np.nan
    left, right = idx[0], idx[-1]
    # linear interp for edges
    x_left  = np.interp(half, y_norm[left-1:left+1], x[left-1:left+1])
    x_right = np.interp(half, y_norm[right:right+2], x[right:right+2])
    return x_right-x_left, x_left, x_right

def process_trace(delays_fs, volts):
    # 1) normalize & baseline
    bg_mask = (delays_fs >= -950) & (delays_fs <= -500)
    bg = np.mean(volts[bg_mask])
    signal = volts/bg - 1.0

    # 2) find only positive, well-spaced peaks
    dt_fs = np.mean(np.diff(delays_fs))
    period_fs = 1e15 / CARRIER_FREQ
    min_dist = int(period_fs/dt_fs)
    peaks, _ = find_peaks(signal,
                          distance=min_dist,
                          prominence=PROMINENCE)
    t_peaks = delays_fs[peaks]
    v_peaks = signal[peaks]

    # 3) cubic interp envelope & Gaussian smooth
    env_raw = interp1d(t_peaks, v_peaks,
                       kind='cubic',
                       fill_value='extrapolate')(delays_fs)
    env_smooth = gaussian_filter1d(env_raw, GAUSS_SIGMA)

    # 4) compute FWHM
    fwhm, lft, rgt = compute_fwhm(delays_fs, env_smooth)
    return env_smooth, fwhm, lft, rgt

# ─── Hardware setup ───────────────────────────────────────────────────────────
# motor
motor = NewportMotor(COM_PORT)
axis  = motor.axis(AXIS)

# DAQ
daq = nidaqmx.Task()
daq.ai_channels.add_ai_voltage_chan(DAQ_CHANNEL)
daq.timing.cfg_samp_clk_timing(rate=DAQ_RATE_HZ,
                               sample_mode=nidaqmx.constants.AcquisitionType.CONTINUOUS,
                               samps_per_chan=POINTS_PER_SCAN)
daq.start()

# ─── Live‐plot setup ─────────────────────────────────────────────────────────
plt.ion()
fig, ax = plt.subplots(figsize=(8,4))
line_env, = ax.plot([], [], 'C1-', lw=2, label='Envelope')
text_f,    = ax.text(0.02, 0.95, '', transform=ax.transAxes,
                     va='top', color='C0')
ax.set_xlabel("Delay (fs)")
ax.set_ylabel("Normalized Voltage")
ax.set_title("Real-time IAC Envelope & FWHM")
ax.legend()

# ─── Main loop ────────────────────────────────────────────────────────────────
direction = +1
while True:
    try:
        # 1) start scan
        if direction>0:
            axis.continuous_scan(START_MM, STOP_MM,
                                 velocity=VELOCITY_MM_S,
                                 poll_interval=POLL_INTERVAL)
        else:
            axis.continuous_scan(STOP_MM, START_MM,
                                 velocity=VELOCITY_MM_S,
                                 poll_interval=POLL_INTERVAL)

        # 2) read back one full trace
        #    (assumes POINTS_PER_SCAN delivered)
        volts = np.array(daq.read(number_of_samples_per_channel=POINTS_PER_SCAN))
        pos   = np.linspace(START_MM if direction>0 else STOP_MM,
                            STOP_MM  if direction>0 else START_MM,
                            POINTS_PER_SCAN)
        delays_fs = pos * (1e15)       # mm→fs conversion factor; adjust if needed

        # 3) process
        env, fwhm, left, right = process_trace(delays_fs, volts)

        # 4) update plot
        line_env.set_data(delays_fs, env)
        ax.set_xlim(delays_fs[0], delays_fs[-1])
        ax.set_ylim(np.min(env), np.max(env)*1.1)
        text_f.set_text(f"FWHM = {fwhm:.1f} fs")
        fig.canvas.draw()
        fig.canvas.flush_events()

        direction *= -1  # reverse next time

    except KeyboardInterrupt:
        print("Stopping real-time scan.")
        break

# ─── Cleanup ─────────────────────────────────────────────────────────────────
daq.stop()
daq.close()
motor.close()


ModuleNotFoundError: No module named 'units'